In [1]:
from core.common import VBTfunc
import vectorbtpro as vbt
import talib
import numpy as np
from numba import njit
import math
import inspect

from core import constants
from core.indicators import VBTKAMA, VBTBBANDSTREND, VBTMACDBBTREND

from trading_bot.settings import STOCH_LL, STOCH_LU, BBAND_THRESHOLD

In [2]:
#only to visualize trend

#modify it depending on what you want to observe
def trend_mode(trend, mode_to_vis ):
    entries=(trend[:]>=mode_to_vis)
    exits=~entries
      
    return entries, exits

VBTTRENDMODE= vbt.IF(
      class_name='VBTPatternEnt',
      short_name='pattern_ent',
      input_names=['macro_trend'],
      param_names=['mode_to_vis'],
      output_names=['entries', 'exits']
 ).with_apply_func(
      trend_mode, 
      takes_1d=True,  
 ) 

In [5]:
class Vis(VBTfunc):
    def __init__(self,period,**kwargs):
        
        self.close_dic={}
        self.open_dic={}
        self.low_dic={}
        self.high_dic={}
        self.indexes=["CAC40", "DAX", "NASDAQ"]
        
        for ind in self.indexes:
            super().__init__(ind,period)
            
            if kwargs.get("index",False):
            #self.only_index=True
                self.close_dic[ind]=self.close_ind
                self.open_dic[ind]=self.open_ind
                self.low_dic[ind]=self.low_ind
                self.high_dic[ind]=self.high_ind
            else:
                self.close_dic[ind]=self.close
                self.open_dic[ind]=self.open
                self.low_dic[ind]=self.low
                self.high_dic[ind]=self.high        
        
        self.trend={}

        for ind in self.indexes:
            if False:
                t=VBTBBANDSTREND.run(self.close_dic[ind])
            else:
                t=VBTMACDBBTREND.run(self.close_dic[ind])
            
            self.trend[ind]=t.trend

    def trend_mode(self,mode_to_vis):
        self.ents={}
        self.exs={}
        
        for ind in self.indexes:
            t=VBTTRENDMODE.run(self.trend[ind],mode_to_vis=mode_to_vis)
            self.ents[ind]=t.entries
            self.exs[ind]=t.exits
            
    def manual_calculate_pf(self,ind, mode_to_vis):
        self.trend_mode(mode_to_vis)
        pf=vbt.Portfolio.from_signals(self.close_dic[ind], self.ents[ind],self.exs[ind],
                                      freq="1d")
        return pf
        

In [6]:
o=Vis("2007_2022") #,index=True

  0%|          | 0/26 [00:00<?, ?it/s]

  0%|          | 0/23 [00:00<?, ?it/s]

  0%|          | 0/69 [00:00<?, ?it/s]

In [7]:
pf=o.manual_calculate_pf("CAC40",10)

In [8]:
#pf.plot()
pf.plot(column=pf.wrapper.columns[3])

FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#1f77b4'},
              'na…

In [11]:
o.trend["CAC40"]["AC"].vbt.plot()

FigureWidget({
    'data': [{'name': 'AC',
              'showlegend': True,
              'type': 'scatter',
…